In [ ]:
#Bu projede ise TF-IDF modeli ile çalışmaya uygun SMS spam veri seti ile çalışacağız

#TF-IDF modeli kelime bazlı  sınıflandırma yaptığı için bert e göre daha hızlı çalışır ve sonuç
#verir



In [ ]:
import pandas as pd

# ; ile ayrıldığı için sep parametresiyle belirtiyoruz
df = pd.read_csv("/content/TurkishSMSCollection.csv", sep=";")

# İlk 5 veriyi göster
df.head()


,Message,Group,GroupText
0,125 lira,2,Normal
1,Baskanin aksam toplantısi fenaymis :),2,Normal
2,Bilal yalçnlara ne zaman gidiyoruz?,2,Normal
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",1,Spam
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,1,Spam


In [ ]:
# Gerekli kütüphaneler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Etiketleri metinle eşle (1 → spam, 2 → ham)
df["label"] = df["Group"].map({1: "spam", 2: "ham"})

# Sadece gerekli sütunları al
df = df[["Message", "label"]]

# Boşlukları temizle
df["Message"] = df["Message"].astype(str).str.strip()
df["label"] = df["label"].astype(str).str.strip()

# Sonucu kontrol et
df.head()


/tmp/ipython-input-29-3094861897.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Message"] = df["Message"].astype(str).str.strip()
/tmp/ipython-input-29-3094861897.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = df["label"].astype(str).str.strip()


,Message,label
0,125 lira,ham
1,Baskanin aksam toplantısi fenaymis :),ham
2,Bilal yalçnlara ne zaman gidiyoruz?,ham
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",spam
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,spam


In [ ]:
X = df["Message"]
y = df["label"]

# %80 eğitim, %20 test verisi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# TF-IDF dönüştürücüyü oluştur
vectorizer = TfidfVectorizer()

# Eğitim ve test verisine uygula
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Modeli oluştur
model = MultinomialNB()

# Eğitimi yap
model.fit(X_train_tfidf, y_train)

# Tahmin yap
y_pred = model.predict(X_test_tfidf)

# Sonuçları değerlendir
print("Doğruluk Oranı:", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))


Doğruluk Oranı: 0.9737118822292324

Sınıflandırma Raporu:
               precision    recall  f1-score   support

         ham       1.00      0.94      0.97       422
        spam       0.95      1.00      0.98       529

    accuracy                           0.97       951
   macro avg       0.98      0.97      0.97       951
weighted avg       0.97      0.97      0.97       951



In [ ]:
def tahmin_et_interaktif():
    while True:
        print("Çıkmak için 'q' yazıp enter'a basabilirsiniz.")
        mesaj = input("📨 E-posta veya SMS içeriğini giriniz: ")

        if mesaj.lower() == 'q':
            print("Sistemden çıkılıyor.")
            break

        mesaj_tfidf = vectorizer.transform([mesaj])
        tahmin = model.predict(mesaj_tfidf)[0]
        print(f"📌 Tahmin Edilen Sınıf: {tahmin}\n")

# Fonksiyonu çalıştır
tahmin_et_interaktif()


Çıkmak için 'q' yazıp enter'a basabilirsiniz.
📨 E-posta veya SMS içeriğini giriniz: Bedava kazandınız! Hemen tıklayın ve ödülünüzü alın!
📌 Tahmin Edilen Sınıf: spam

Çıkmak için 'q' yazıp enter'a basabilirsiniz.
📨 E-posta veya SMS içeriğini giriniz: Kredi kartı borcunuz silinebilir. Hemen bize ulaşın!
📌 Tahmin Edilen Sınıf: spam

Çıkmak için 'q' yazıp enter'a basabilirsiniz.
📨 E-posta veya SMS içeriğini giriniz: Tamam, akşam 8 gibi buluşuruz
📌 Tahmin Edilen Sınıf: ham

Çıkmak için 'q' yazıp enter'a basabilirsiniz.
📨 E-posta veya SMS içeriğini giriniz: Bugün ofiste çok yoğundum, biraz geç yazabildim.
📌 Tahmin Edilen Sınıf: ham

Çıkmak için 'q' yazıp enter'a basabilirsiniz.
📨 E-posta veya SMS içeriğini giriniz: q
Sistemden çıkılıyor.
